In [157]:
import pandas as pd
import json
employee_raw_data = pd.read_excel("database\DanhSachThongTinDoanVien.xlsx")
main_data = employee_raw_data.iloc[5:]

In [158]:
mapping_cols = {
    "stt": "Unnamed: 0",
    "maso_doanvien": "Unnamed: 1",
    "hovaten": "Unnamed: 2",
    "ngaysinh_nam": "Unnamed: 3",
    "ngaysinh_nu": "Unnamed: 4",
    "tinh": "Unnamed: 5",
    "matinh": "Unnamed: 6",
    "cmnd": "Unnamed: 7",
    "huongluong_ngansach": "Unnamed: 8",
    "huongluong_ngoaingansach": "Unnamed: 9",
    "khongchuyentrach": "Unnamed: 10",
    "chutich": "Unnamed: 11",
    "phochutich": "Unnamed: 12",
    "uvbch": "Unnamed: 13",
    "totruong": "Unnamed: 14",
    "topho": "Unnamed: 15",
    "cb_cdcs": "Unnamed: 16",
    "vuvbkt": "Unnamed: 17",
    "ngayvao_congdoan": "Unnamed: 18",
    "Unnamed_19": "Unnamed: 19",
    "Unnamed_20": "Unnamed: 20",
    "Unnamed_21": "Unnamed: 21",
    "Unnamed_22": "Unnamed: 22",
    "nguyenquan_tinh": "Unnamed: 23",
    "nguyenquan_matinh": "Unnamed: 24",
}
mapping_cols_swap = {v: k for k, v in mapping_cols.items()}

In [159]:
import numpy as np
from itertools import product

In [160]:
def joining_date_by_gender(length, main_data, mapping_cols_swap):
    result =  main_data.iloc[:int(length)]
    new_result = result.rename(columns=mapping_cols_swap)
    new_result = new_result.reset_index(drop=True)
    union_joining_date_by_gender_df = new_result[["ngayvao_congdoan", "ngaysinh_nu", "ngaysinh_nam"]]
    gender = []
    joining_date = []
    union_joining_date_by_gender_df = union_joining_date_by_gender_df.fillna("")
    for irow in range(len(union_joining_date_by_gender_df)):
        row  = union_joining_date_by_gender_df.iloc[irow]
        if row["ngaysinh_nu"] == "":
            gender.append("nam")
        else:
            gender.append("nu")
        joining_year = row["ngayvao_congdoan"].split("/")[-1]
        joining_date.append(int(joining_year))
    union_joining_date_by_gender_df["gender"] = gender
    union_joining_date_by_gender_df["joining_date"] = joining_date
    final_df = union_joining_date_by_gender_df[["gender", "joining_date"]]
    final_df = pd.DataFrame({'value' : final_df.groupby( ["gender", "joining_date"] ).size()}).reset_index()
    final_df = final_df.sort_values("joining_date")
    df = final_df.copy()
    

    nam_df = df[df['gender']=='nam']
    nam_df = nam_df.rename(columns={'value':'nam_value'})
    nam_df = nam_df.drop(columns=['gender'])

    # create a dataframe for nu
    nu_df = df[df['gender']=='nu']
    nu_df = nu_df.rename(columns={'value':'nu_value'})
    nu_df = nu_df.drop(columns=['gender'])

    # merge nam and nu dataframes on joining_date
    merged_df = pd.merge(nam_df, nu_df, on='joining_date', how='outer')

    # create a new column 'tong_value' as the sum of 'nam_value' and 'nu_value'
    merged_df['tong_value'] = merged_df['nam_value'].fillna(0) + merged_df['nu_value'].fillna(0)

    # create a new dataframe for 'tong'
    tong_df = merged_df[['joining_date', 'tong_value']]
    tong_df = tong_df.rename(columns={'tong_value':'value'})
    tong_df['gender'] = 'tong'

    # append 'tong' dataframe to the original dataframe
    df = df.append(tong_df, ignore_index=True)

    # sort the dataframe by gender, joining_date
    df = df.sort_values('joining_date').reset_index(drop=True)

    # fill missing values with 0
    df['value'] = df['value'].fillna(0)

    # create a new DataFrame with all possible combinations of gender and year
    all_genders = df['gender'].unique()
    min_year = df['joining_date'].min()
    max_year = df['joining_date'].max()
    year_range = range(min_year, max_year+1)
    all_years = pd.DataFrame(list(product(all_genders, year_range)), columns=['gender', 'joining_date'])

    # merge with the original DataFrame to fill missing values with 0
    result = pd.merge(df, all_years, how='right', on=['gender', 'joining_date'])
    result['value'] = result['value'].fillna(0)

    # sort the result by gender and joining_date
    result.sort_values(['gender', 'joining_date'], inplace=True)

    # reset the index
    result.reset_index(drop=True, inplace=True)
    result = result.sort_values('joining_date').reset_index(drop=True)
    result = result[result["gender"]!= "tong"]
    return result.to_json(orient="records")

In [ ]:
x = joining_date_by_gender(-1, main_data, mapping_cols_swap)
x

In [142]:
import datetime
def cal_age(length, main_data, mapping_cols_swap):
    length = -1
    result =  main_data.iloc[:int(length)]
    new_result = result.rename(columns=mapping_cols_swap)
    new_result = new_result.reset_index(drop=True)
    date_gender_df = new_result[["ngaysinh_nu", "ngaysinh_nam"]]
    gender = []
    born = []
    age = []
    date_gender_df = date_gender_df.fillna("")
    for irow in range(len(date_gender_df)):
        row  = date_gender_df.iloc[irow]
        byear = ''
        if row["ngaysinh_nam"] != "":
            gender.append("nam")
            byear = int(row["ngaysinh_nam"].split("/")[-1])
            born.append(byear)
        if row["ngaysinh_nu"] != "":
            gender.append("nu")
            byear = int(row["ngaysinh_nu"].split("/")[-1])
            born.append(byear)
        age.append(str(int(datetime.datetime.now().year) - byear))
    date_gender_df["gender"] = gender
    date_gender_df["born"] = born
    date_gender_df["age"] = age
    date_gender_df = date_gender_df.drop(columns=["ngaysinh_nam", "ngaysinh_nu", "gender"])
    result_df = date_gender_df.drop(columns=["born"])
    result_df = result_df.value_counts("age").reset_index()
    result_df.columns = ['age', 'value']
    result_df = result_df.sort_values("age")
    return result_df.to_json(orient="records")



In [156]:
def province_distribution(length, main_data, mapping_cols_swap, top = -1):
    result =  main_data.iloc[:int(length)]
    new_result = result.rename(columns=mapping_cols_swap)
    new_result = new_result.reset_index(drop=True)
    result_df = new_result.value_counts("tinh").reset_index()
    result_df.columns = ['province', 'value']
    result_df = result_df.sort_values("value", ascending = False)
    if top != -1:
        result_df = result_df[:top]
    return result_df.to_json(orient="records")


In [ ]:
result =  main_data.iloc[:int(length)]
new_result = result.rename(columns=mapping_cols_swap)
new_result = new_result.reset_index(drop=True)
result_df = new_result.value_counts("tinh").reset_index()
result_df.columns = ['province', 'value']
result_df = result_df.sort_values("value", ascending = False)
result_df[:10]
# return result_df.to_json(orient="records")

In [167]:
length = -1
result =  main_data.iloc[:int(length)]
new_result = result.rename(columns=mapping_cols_swap)
new_result = new_result.reset_index(drop=True)


gender = []
born = []
age = []
date_gender_df = new_result.fillna("")
for irow in range(len(date_gender_df)):
    row  = date_gender_df.iloc[irow]
    byear = ''
    if row["ngaysinh_nam"] != "":
        gender.append("nam")
        byear = row["ngaysinh_nam"]
        born.append(byear)
    if row["ngaysinh_nu"] != "":
        gender.append("nu")
        byear = row["ngaysinh_nu"]
        born.append(byear)

date_gender_df["gioitinh"] = gender
date_gender_df["ngaysinh"] = born

date_gender_df = date_gender_df.drop(columns=["ngaysinh_nam", "ngaysinh_nu"])

In [169]:
date_gender_df.to_excel("clean_DanhSachThongTinDoanVien.xlsx")

# Event

In [1]:
import pandas as pd

In [74]:
event_data = pd.read_excel("database/event_db.xlsx", dtype=str)

In [75]:
event_data = event_data.fillna("")

In [76]:
requested_event = event_data[event_data["requesting_member_id"].str.contains("19031998", na=False)]
requested_event

,event_id,event_name,event_description,event_start_date,event_end_date,requesting_member_id,accepted_member_id,rejected_member_id
0,0001,Hội thao 2023,Đây là mô tả của sự kiện hội thao năm 2023,05/04/2023,15/04/2023,"079091006696, 072068001015, 19031998","079091006696, 072068001015, 19031998",
1,0002,Văn nghệ 2023,Cuộc thi văn nghệ tổ chức vào dịp 30/4 - 1/5 2023,20/04/2023,25/04/2023,"079091006696, 072068001015, 19031998","079091006696, 072068001015",
2,0003,Sự kiện 2024,Đây là mô tả của sự kiện 2024,20/12/2023,05/01/2024,"072068001015, 19031998",,19031998
3,0004,Demoday,"Demo dự án, cập nhật tiến độ",20/03/2023,20/03/2023,19031998,19031998,


In [77]:
accepted_event = requested_event[requested_event["accepted_member_id"].str.contains("19031998", na=False)]
accepted_event

,event_id,event_name,event_description,event_start_date,event_end_date,requesting_member_id,accepted_member_id,rejected_member_id
0,0001,Hội thao 2023,Đây là mô tả của sự kiện hội thao năm 2023,05/04/2023,15/04/2023,"079091006696, 072068001015, 19031998","079091006696, 072068001015, 19031998",
3,0004,Demoday,"Demo dự án, cập nhật tiến độ",20/03/2023,20/03/2023,19031998,19031998,


In [78]:
rejected_event = requested_event[requested_event["rejected_member_id"].str.contains("19031998", na=False)]
rejected_event

,event_id,event_name,event_description,event_start_date,event_end_date,requesting_member_id,accepted_member_id,rejected_member_id
2,0003,Sự kiện 2024,Đây là mô tả của sự kiện 2024,20/12/2023,05/01/2024,"072068001015, 19031998",,19031998


In [79]:
requested_event.index

Int64Index([0, 1, 2, 3], dtype='int64')

In [81]:
pendding_indexes = [each for each in list(requested_event.index) if each not in (list(accepted_event.index) + list(rejected_event.index))]

In [82]:
list(accepted_event.index)

[0, 3]

In [83]:
list(rejected_event.index)

[2]

In [84]:
pendding_event = requested_event.loc[pendding_indexes]
pendding_event

,event_id,event_name,event_description,event_start_date,event_end_date,requesting_member_id,accepted_member_id,rejected_member_id
1,0002,Văn nghệ 2023,Cuộc thi văn nghệ tổ chức vào dịp 30/4 - 1/5 2023,20/04/2023,25/04/2023,"079091006696, 072068001015, 19031998","079091006696, 072068001015",


In [40]:
temp_df = requested_event.copy()
temp_df = temp_df.set_index("event_id")
short_event_data = temp_df.to_dict()
requested_event_list = requested_event["event_id"].to_list()
accepted_event_list = accepted_event["event_id"].to_list()
rejected_event_list = rejected_event["event_id"].to_list()

result = {
    # "requested_event_list": requested_event_list,
    "accepted_event_list": accepted_event_list,
    "rejected_event_list": rejected_event_list,
    "short_event_data": short_event_data,
}

print(result)

{'requested_event_list': [1, 2, 3], 'accepted_event_list': [1], 'rejected_event_list': [3], 'short_event_data': {'event_name': {1: 'Hội thao 2023', 2: 'Văn nghệ 2023', 3: 'Sự kiện 2024'}, 'event_description': {1: 'Đây là mô tả của sự kiện hội thao năm 2023', 2: 'Cuộc thi văn nghệ tổ chức vào dịp 30/4 - 1/5 2023', 3: 'Đây là mô tả của sự kiện 2024'}, 'event_start_date': {1: '05/04/2023', 2: '20/04/2023', 3: '20/12/2023'}, 'event_end_date': {1: '15/04/2023', 2: '25/04/2023', 3: '05/01/2024'}, 'requesting_member_id': {1: '079091006696, 072068001015, 19031998', 2: '079091006696, 072068001015, 19031998', 3: '072068001015, 19031998'}, 'accepted_member_id': {1: '079091006696, 072068001015, 19031998', 2: '079091006696, 072068001015', 3: ''}, 'rejected_member_id': {1: '', 2: '', 3: "'19031998"}}}


In [ ]:
requested_event

,event_id,event_name,event_description,event_start_date,event_end_date,requesting_member_id,accepted_member_id,rejected_member_id
0,1,Hội thao 2023,Đây là mô tả của sự kiện hội thao năm 2023,05/04/2023,15/04/2023,"079091006696, 072068001015, 19031998","079091006696, 072068001015, 19031998",
1,2,Văn nghệ 2023,Cuộc thi văn nghệ tổ chức vào dịp 30/4 - 1/5 2023,20/04/2023,25/04/2023,"079091006696, 072068001015, 19031998","079091006696, 072068001015",
